# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [3]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
    

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_new csv that
# will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
import pandas as pd
event = pd.read_csv('event_datafile_new.csv')
event.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster


# To establish connection and begin executing queries, need a session
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'3




## Query 1:
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

__________________________________________________________________________________________________________________
### Answer

#### Table Description
The given task states that we obtain first and last name of every user in the music app that listened to "All hands against his own"

#### What kind of features do we need?
The table will consists of;
1. artist
2. ItemInSession
3. length
4. level
5. sessionId
6. song

The primary key for this table will be "sessionId" and "itemInSession" 
1. "sessionId" will be the partition key
2. itemInSession will be the clustering column due to the uniquessness of itemInSession

In [13]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_library 
    (artist text, 
    itemInSession int, 
    length float , 
    level text , 
    sessionId int , 
    song text , 
    PRIMARY KEY (sessionId, itemInSession))
    """)
except Exception as e:
    print("Error creating table")
    print(e)

#### Note! Be sure to take note of the oreder within the partion,  The partiton and clustering keys must be displayed first

In [14]:
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

1. First we create a function called "select" for perfroming the select task
3. Next we call the select fucntion and pass our queries

In [15]:
def select(query):
    """Execute a query to select specific table values"""
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print (list(row))

In [17]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
select(("SELECT artist, song, length FROM song_library WHERE sessionId=338 AND itemInSession=4"))

['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


## Query 2:
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
for userid = 10, sessionid = 182

In [27]:
try:
    session.execute("""
   CREATE TABLE IF NOT EXISTS song_library_by_user(
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
    """)
except Exception as e:
    print("Error creating table")
    print(e)

#### Note! Be sure to take note of the oreder within the partion,  The partiton and clustering keys must be displayed first

In [21]:
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### To test our tables,  lets perform a select operation to print out a particular result 

In [22]:
select("SELECT sessionId, artist, firstName, lastName FROM song_library_by_user WHERE userId=10 AND sessionId=182")

[182, 'Down To The Bone', 'Sylvie', 'Cruz']
[182, 'Three Drives', 'Sylvie', 'Cruz']
[182, 'Sebastien Tellier', 'Sylvie', 'Cruz']
[182, 'Lonnie Gordon', 'Sylvie', 'Cruz']


## Query 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


__________________________________________________________________________________________________________________
### Answer

#### Table Description
The given task states that we obtain first and last name of every user in the music app that listened to "All hands against his own"

#### What kind of features do we need?
The table will consists of;
1. First Name
2. Last Name
3. User ID
4. Song

The primary key for this table will be "Song" and UserId, where 
1. "Song" will be the partition key
2. userId will be the clustering column due to the unniqueness of the Song column


In [23]:
try:
    session.execute("""
   CREATE TABLE IF NOT EXISTS song_library_by_song(
        firstName text,
        lastName text,
        userId int,
        song text,
        PRIMARY KEY (song, userId)
    )
    """)
except Exception as e:
    print("Error creating table")
    print(e)
    

#### Note! Be sure to take note of the oreder within the partion,  The partiton and clustering keys must be displayed first

In [24]:
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library_by_song (firstName, lastName,userId ,song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))
      

#### To test our tables,  lets perform a select operation to print out a particular result 

In [25]:
select("SELECT firstName, lastName FROM song_library_by_song WHERE song='All Hands Against His Own'")

['Jacqueline', 'Lynch']
['Tegan', 'Levine']
['Sara', 'Johnson']


### Drop the tables before closing out the sessions

In [25]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_library")
session.execute("DROP TABLE IF EXISTS song_library_by_user")
session.execute("DROP TABLE IF EXISTS song_library_by_song")

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()

In [28]:
!tar cvfz correctfiles.tar.gz *

apache_cassandra.ipynb
event_data/
event_data/2018-11-30-events.csv
event_data/2018-11-23-events.csv
event_data/2018-11-22-events.csv
event_data/2018-11-29-events.csv
event_data/2018-11-11-events.csv
event_data/2018-11-14-events.csv
event_data/2018-11-20-events.csv
event_data/2018-11-15-events.csv
event_data/2018-11-05-events.csv
event_data/2018-11-28-events.csv
event_data/2018-11-25-events.csv
event_data/2018-11-16-events.csv
event_data/2018-11-18-events.csv
event_data/2018-11-24-events.csv
event_data/2018-11-04-events.csv
event_data/2018-11-19-events.csv
event_data/2018-11-26-events.csv
event_data/2018-11-12-events.csv
event_data/2018-11-27-events.csv
event_data/2018-11-06-events.csv
event_data/2018-11-09-events.csv
event_data/2018-11-03-events.csv
event_data/2018-11-21-events.csv
event_data/2018-11-07-events.csv
event_data/2018-11-01-events.csv
event_data/2018-11-13-events.csv
event_data/2018-11-17-events.csv
event_data/2018-11-08-events.csv
event_data/2018-11-10-events.csv
event_da

__________________________________________________________________________________________________________________
### Answer

#### Table Description
The given task states that we obtain first and last name of every user in the music app that listened to "All hands against his own"

#### What kind of features do we need?
The table will consists of;
1. UserId
2. SessionId
3. ItemInSession
4. artist
5. firstName
6. lastName

The primary key for this table will be "userId" and "SessionId" and "itemInSession" 
1. "userId" will be the partition key
2. SessionId and itemInSession will be the clustering column
